<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/attrition.png?raw=true" width=800 align="left"/>

# 실습 내용

- 다양한 알고리즘으로 모델을 만들고 성능을 검증합니다.
- 성능이 좋을 것으로 판단된 모델로 예측 및 평가를 수행합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/Attrition_simple2.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [ ]:
# 데이터 살펴보기
data.head()

**데이터 설명**

- Attrition: 이직 여부 (1: 이직, 0: 잔류)
- Age: 나이
- DistanceFromHome: 집-직장 거리 (단위: 마일)
- EmployeeNumber: 사번
- Gender: 성별 (Male, Female)
- JobSatisfaction: 직무 만족도(1: Low, 2: Medium, 3: High, 4: Very High)
- MaritalStatus: 결혼 상태 (Single, Married, Divorced)
- MonthlyIncome: 월급 (단위: 달러)
- OverTime: 야근 여부 (Yes, No)
- PercentSalaryHike: 전년 대비 급여 인상율(단위: %)
- TotalWorkingYears: 총 경력 연수

In [ ]:
# 기술통계 확인
data.describe()

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) 변수 제거**

- 제거 대상 변수: EmployeeNumber

In [ ]:
# 제거 대상: EmployeeNumber
del_cols = ['EmployeeNumber']

# 변수 제거
data.drop(del_cols, axis=1, inplace=True)

# 확인
data.head()

**2) x, y 분리**

In [ ]:
# target 확인
target = 'Attrition'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**3) 가변수화**

In [ ]:
# 가변수화 대상
dumm_cols = ['Gender', 'JobSatisfaction', 'MaritalStatus', 'OverTime']

# 가변수화
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True, dtype=int)

# 확인
x.head()

**4) 학습용, 검증용, 평가용 데이터 분리**

- 학습용, 평가용 데이터는 7:3으로 분리합니다.
- 학습 데이터를 다시 8:2 비율로 학습용, 검증용 데이터로 분리합니다.

In [ ]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 학습용, 평가용 7:3 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

# 학습용, 검증용 8:2 분리
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

**5) 정규화**

- KNN 알고리즘을 위해 정규화를 진행하여 별도로 저장합니다.
- 평가 데이터에 대한 정규화는 진행하지 않습니다.
- 평가 데이터에 대한 정규화는 최종 평가 시 필요할 때 진행합니다.

In [ ]:
# 모듈 불러오기
from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_val_s = scaler.transform(x_val)

# 4.성능 예측

- 여러 알고리즘으로 모델을 만들고 K-Fold CV로 성능을 예측합니다.
- 하이퍼파라미터를 설정하지 않고 진행합니다.
- 각 모델의 성능 정보를 수집해 마지막에 비교합니다.

In [ ]:
# xgboost 설치
# !pip install xgboost

In [ ]:
# lightgbm 설치
# !pip install lightgbm

- 이후 사용할 함수를 모두 불러옵니다.

In [ ]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *

**1) KNN**

- KNN 알고리즘으로 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

In [ ]:
# 매개변수 범위('n_neighbors': range(1, 21))
params = {'n_neighbors': range(1, 21)}

# 모델 선언
model_knn = GridSearchCV(KNeighborsClassifier(), params, cv=5)

# 학습
model_knn.fit(x_train_s, y_train)

# 매개변수와 성능 확인
print(model_knn.best_params_)
print(model_knn.best_score_)

- 검증 데이터로 성능을 검증하고, 검증된 성능 정보를 수집해 다른 모델과 비교합니다.

In [ ]:
# 예측하기
y_val_pred = model_knn.predict(x_val_s)

# 검증하기
print(confusion_matrix(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

# 성능정보 수집(Accuracy Score)
result = {}
result['KNN'] = accuracy_score(y_val, y_val_pred)
print(result)

**2) Decision Tree**

- Decision Tree 알고리즘으로 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

In [ ]:
# 매개변수 범위('max_depth': range(1, 11))
params = {'max_depth': range(1, 11)}

# 모델 선언
model_dt = GridSearchCV(DecisionTreeClassifier(), params, cv=5)

# 학습
model_dt.fit(x_train, y_train)

# 매개변수와 성능 확인
print(model_dt.best_params_)
print(model_dt.best_score_)

- 검증 데이터로 성능을 검증하고, 검증된 성능 정보를 수집해 다른 모델과 비교합니다.

In [ ]:
# 예측하기
y_val_pred = model_dt.predict(x_val)

# 검증하기
print(confusion_matrix(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

# 성능정보 수집(Accuracy Score)
result['Decision Tree'] = accuracy_score(y_val, y_val_pred)
print(result)

**3) Random Forest**

- Random Forest 알고리즘으로 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

In [ ]:
# 매개변수 범위('max_depth': range(1, 11), 'n_estimators': range(80, 121, 10))
params = {'max_depth': range(1, 11), 'n_estimators': range(80, 121, 10)}

# 모델 선언
model_rf = GridSearchCV(RandomForestClassifier(), params, cv=5)

# 학습
model_rf.fit(x_train, y_train)

# 매개변수와 성능 확인
print(model_rf.best_params_)
print(model_rf.best_score_)

- 검증 데이터로 성능을 검증하고, 검증된 성능 정보를 수집해 다른 모델과 비교합니다.

In [ ]:
# 예측하기
y_val_pred = model_rf.predict(x_val)

# 검증하기
print(confusion_matrix(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

# 성능정보 수집(Accuracy Score)
result['Random Forest'] = accuracy_score(y_val, y_val_pred)
print(result)

**4) LightGBM**

- LightGBM 알고리즘으로 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

In [ ]:
# 매개변수 범위('max_depth': range(1, 11), 'n_estimators': range(80, 121, 10))
params = {'max_depth': range(1, 11), 'n_estimators': range(80, 121, 10)}

# 모델 선언
model_lgb = GridSearchCV(LGBMClassifier(importance_type='gain', verbose=-1), params, cv=5)

# 학습
model_lgb.fit(x_train, y_train)

# 매개변수와 성능 확인
print(model_lgb.best_params_)
print(model_lgb.best_score_)

- 검증 데이터로 성능을 검증하고, 검증된 성능 정보를 수집해 다른 모델과 비교합니다.

In [ ]:
# 예측하기
y_val_pred = model_lgb.predict(x_val)

# 검증하기
print(confusion_matrix(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

# 성능정보 수집(Accuracy Score)
result['LightGBM'] = accuracy_score(y_val, y_val_pred)
print(result)

# 5.성능 비교

- 검증된 각 모델의 성능을 비교합니다.

In [ ]:
# 데이터프레임 만들기
score = pd.DataFrame()
score['Model'] = list(result)
score['Accuracy'] = result.values()

# 확인
score

# 6.최종 예측 및 평가

- 가장 성능이 좋다고 검증된 모델로 평가 데이터에 대해 예측 및 평가를 수행합니다.

In [ ]:
# 예측하기
y_pred = model_lgb.predict(x_test)

In [ ]:
# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

- 변수 중요도를 시각화합니다.

In [ ]:
# 데이터프레임 만들기
df = pd.DataFrame()
df['Feature'] = list(x)
feature_importances_gain = model_lgb.best_estimator_.feature_importances_
feature_importances_normalized = feature_importances_gain / np.sum(feature_importances_gain)
df['Importance'] = feature_importances_normalized
df.sort_values(by='Importance', ascending=True, inplace=True)

# 시각화
plt.figure(figsize=(3, 5))
plt.barh(y=df['Feature'], width=df['Importance'])
plt.show()